# Goal

As identified in the previous notebook, the `DecisionTree` gives the best performance on the validation set and hence we use it on All the Features and also perform ablations (not in this notebook) by excluding groups of features.


## Imports


In [1]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from skimage.feature import local_binary_pattern
from skimage.color import rgb2lab, rgb2hsv
import pywt
import pandas as pd
from PIL import Image
from scipy.stats import spearmanr, pearsonr
from scipy import ndimage
from skimage.feature import hog, graycomatrix, graycoprops
from sklearn.tree import DecisionTreeRegressor
from pathlib import Path
import numpy as np
from PIL import Image

import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)


## Reading the preparing the data

In [2]:
train_df = pd.read_csv("../../dataset/ratings/train.csv")
test_df = pd.read_csv("../../dataset/ratings/test.csv")
val_df = train_df.iloc[-90:].copy()
train_df = train_df.iloc[:-90].copy()
print(f"val_df.shape: {val_df.shape}")
print(f"train_df.shape: {train_df.shape}")


val_df.shape: (90, 3)
train_df.shape: (420, 3)


In [3]:
train_df.head()


,filename,MOS_Rating,MOS_ZScore
0,f329.png,66.346670,0.134646
1,f316.png,47.456209,-0.954147
2,f257.png,80.986020,0.978417
3,f206.png,11.237613,-3.041686
4,f459.png,60.666032,-0.192770


In [4]:
val_df.head()


,filename,MOS_Rating,MOS_ZScore
420,f68.png,61.416769,-0.149500
421,f191.png,59.814871,-0.241829
422,f278.png,66.696317,0.154798
423,f62.png,72.154690,0.469404
424,f164.png,75.020202,0.634564


In [5]:
test_df.head()


,filename,MOS_Rating,MOS_ZScore
0,f244.png,75.843124,0.681995
1,f126.png,76.323762,0.709697
2,r3.png,85.236324,1.223393
3,f145.png,42.999260,-1.211034
4,f170.png,51.925539,-0.696548


In [6]:
X_train = list(train_df["filename"])
y_train = list(train_df["MOS_ZScore"])

X_valid = list(val_df["filename"])
y_valid = list(val_df["MOS_ZScore"])

X_test = list(test_df["filename"])
y_test = list(test_df["MOS_ZScore"])


## Defining `FeatureExtractor`:


In [7]:
class FeatureExtractor:
    def __init__(self, img_dir=""):
        self.img_dir = img_dir

    # ------------------------------------------------------------------ #
    # --------------------  feature extraction ------------------------- #
    # ------------------------------------------------------------------ #
    def extract(self, image_name):
        img = np.asarray(Image.open(self.img_dir + image_name))
        gray = np.mean(img, axis=2)
        h, w = gray.shape

        feats = []


        # — Color features
        # ---------- 1. Enhanced colour features (8) ----------
        lab = rgb2lab(img)
        hsv = rgb2hsv(img)
        lab_a = lab[:, :, 1].astype(np.float32)/127
        lab_b = lab[:, :, 2].astype(np.float32)/127
        feats.extend([
            np.std(lab_a), np.std(lab_b),
            hsv[:,:,0].std(),
            np.percentile(hsv[:,:,1],95),
            np.mean(np.abs(lab_a)), np.mean(np.abs(lab_b)),
            np.median(hsv[:,:,2]),
            (hsv[:,:,2]>0.9).mean()
        ])



        # — Edge features
        # ---------- 2. Advanced edge features (10) ----------
        sx = ndimage.sobel(gray, axis=1)
        sy = ndimage.sobel(gray, axis=0)
        mag = np.hypot(sx,sy)
        feats.extend([
            mag.mean(), mag.std(),
            np.percentile(mag,90),
            (mag>0.2).mean(),
            np.corrcoef(sx[::10].flatten(), sy[::10].flatten())[0,1],
            np.std(np.arctan2(sy,sx)[mag>0.1]),
            np.abs(sx - np.fliplr(sx)).mean(),
            np.abs(sy - np.flipud(sy)).mean(),
            np.mean(mag[mag>0.05]), mag.max()
        ])


        # — Texture (GLCM + HOG)
        # ---------- 3. Texture & pattern (15) ----------
        glcm = graycomatrix((gray*255).astype(np.uint8),
                            distances=[1,3],
                            angles=[0,np.pi/4],
                            levels=256, symmetric=True)
        for prop in ["contrast","dissimilarity","homogeneity","energy","correlation"]:
            feats.extend(graycoprops(glcm, prop).flatten())
        hogf = hog(gray, orientations=8,
                   pixels_per_cell=(32,32),
                   cells_per_block=(1,1),
                   visualize=False)
        feats.extend([hogf.mean(), hogf.std()])



        # — Frequency (wavelets)
        # ---------- 4. Frequency & compression (12) ----------
        coeffs = pywt.wavedec2(gray, 'db2', level=3)
        for cH,cV,cD in coeffs[1:]:
            feats.extend([
                np.abs(cH).mean(),
                np.abs(cV).mean(),
                np.abs(cD).mean(),
                np.percentile(np.abs(cH), 90)
            ])



        # ---------- 5. Perceptual grouping (two extras) ----------
        ce = ndimage.gaussian_filter(gray,1) - ndimage.gaussian_filter(gray,3)
        feats.append(np.mean(ce**2))
        sal = np.abs(ndimage.gaussian_gradient_magnitude(gray,2))
        feats.extend([sal.mean(), sal.std()])

        return np.array(feats)

    def transform(self, image_list):
        return np.vstack([self.extract(img) for img in image_list])


### Extract features for train/val/test


In [8]:
fe = FeatureExtractor(img_dir="../../dataset/images/train_images/")
X_train_feats = fe.transform(X_train)
X_valid_feats = fe.transform(X_valid)

fe = FeatureExtractor(img_dir="../../dataset/images/test_images/")
X_test_feats  = fe.transform(X_test)


## Experiment (train and test the model)


### Train and Validation


In [9]:
models = {
    "DecisionTree":          DecisionTreeRegressor(max_depth=5, random_state=SEED),
}

# -------------------------------------------------------------------
#  4) Experiment: train on train, validate on val
# -------------------------------------------------------------------
best_name    = None
best_spearman = -np.inf
best_model   = None
results = {}

for name, model in models.items():
    # 1) fit on train
    model.fit(X_train_feats, y_train)

    # 2) score on val
    preds_val = model.predict(X_valid_feats)
    spearman_val, _ = spearmanr(preds_val, y_valid)
    pearson_val,  _ = pearsonr(preds_val, y_valid)

    results[name] = {"spearman": spearman_val, "pearson": pearson_val}
    print(f"{name:20s} → Spearman: {spearman_val:.4f}, Pearson: {pearson_val:.4f}")

    # 3) track best by Spearman on val
    if spearman_val > best_spearman:
        best_spearman = spearman_val
        best_name     = name
        best_model    = model


DecisionTree         → Spearman: 0.5331, Pearson: 0.4895


### Test Prediction


In [10]:
# -------------------------------------------------------------------
#  5) Evaluate the selected best model on test set
# -------------------------------------------------------------------
preds_test = best_model.predict(X_test_feats)
spearman_test, _ = spearmanr(preds_test, y_test)
pearson_test,  _ = pearsonr(preds_test, y_test)

print(f"Test performance of {best_name}:")
print(f"  Spearman: {spearman_test:.4f}")
print(f"  Pearson : {pearson_test:.4f}")


Test performance of DecisionTree:
  Spearman: 0.5033
  Pearson : 0.4339
